In [40]:
!python -V

Python 3.10.12


In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
cd '/content/drive/MyDrive/Notebooks'

/content/drive/MyDrive/Notebooks


In [43]:
import pandas as pd

In [44]:
import pickle

In [45]:
import seaborn as sns
import matplotlib.pyplot as plt

In [46]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [117]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

In [118]:
# Q1: How many columns are there?
len(df.columns)

19

In [119]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [120]:
# Q2: standard deviation
df['duration'].std()

42.59435124195458

In [82]:
df_filtered = df.query('duration > 1 and duration < 60')

In [83]:
# Q3: What fraction of the records left after you dropped the outliers?
len(df_filtered)/len(df)

0.9811146334607858

In [84]:
df = df_filtered

In [85]:
df_val = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)
df_val = df_val.query('duration > 1 and duration < 60')

In [93]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)
df_val[categorical] = df_val[categorical].astype(str)

In [94]:
df_all = pd.concat([df, df_val], ignore_index=True)  # Combine training and validation data
dv = DictVectorizer()
dv.fit(df_all[categorical].to_dict(orient='records'))

DictVectorizer()

In [105]:
train_dicts = df[categorical].to_dict(orient='records')
X_train = dv.transform(train_dicts)

In [109]:
# Q4: One-hot encoding
X_train.shape

(3008849, 514)

In [110]:
target = 'duration'
y_train = df[target].values

In [111]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

# Q5: What's the RMSE on train?
mean_squared_error(y_train, y_pred, squared=False)

7.647676829002479

In [112]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [113]:
X_val.shape

(2855558, 514)

In [114]:
target = 'duration'
y_val = df_val[target].values

In [115]:
y_pred = lr.predict(X_val)

# Q6: What's the RMSE on validation?
mean_squared_error(y_val, y_pred, squared=False)

7.808379477361537